In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)

### First concept: the Model

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [ ]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Second concept: The Message

In [4]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

TextMessage(id='8b450f32-1d16-43da-80d9-49d9ef764338', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 30, 11, 33, 40, 435332, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

### Third concept: The Agent

In [5]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

### Put it all together with on_messages

In [6]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'Great choice! Just remember: if it rains, it’s just the sky’s way of saying hello!'

### Let's make a local database of ticket prices

In [7]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [8]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [9]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [10]:
get_city_price("Rome")

499.0

In [11]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

In [12]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_7JK5nc3n5ezkZ7qJznk7Tnox', arguments='{"city_name":"London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='call_7JK5nc3n5ezkZ7qJznk7Tnox', is_error=False)]


'A trip to London? Fantastic choice! A roundtrip ticket will cost you around $299. Just remember, the only thing more expensive than a cup of tea there is the sincerity of the locals! 🫖✈️'